# Metadata Filtering
- This notebook aims to demonstrate metadata filtering with a simple example using `Goodreads dataset`

In [ ]:
from db_connection import create_db_connection
from tabulate import tabulate

import pandas as pd

In [25]:
conn = create_db_connection()

In [ ]:
with conn.cursor() as cur:
    cur.execute("CREATE EXTENSION IF NOT EXISTS aidb cascade;")
    cur.execute("CREATE EXTENSION IF NOT EXISTS pgfs;")
    cur.execute("DROP TABLE IF EXISTS books;")
    cur.execute("""CREATE TABLE books (
    bookID INT,
    title TEXT,
    authors TEXT,
    average_rating FLOAT,
    isbn TEXT,
    isbn13 BIGINT,
    language_code TEXT,
    num_pages INT,
    ratings_count INT,
    text_reviews_count INT,
    publication_date DATE,
    publisher TEXT
);""")
conn.commit()

In [17]:
# Read the train.csv file into a pandas dataframe, skipping bad lines
df = pd.read_csv('sample_data/books.csv', on_bad_lines="skip")

In [35]:
df.columns

Index(['bookID', 'title', 'authors', 'average_rating', 'isbn', 'isbn13',
       'language_code', 'num_pages', 'ratings_count', 'text_reviews_count',
       'publication_date', 'publisher'],
      dtype='object')

In [19]:
from io import StringIO
# Create a string buffer
output = StringIO()
df_copy = df.copy()

# Convert data types and handle NULL values
df_copy['bookID'] = pd.to_numeric(df_copy['bookID'], errors='coerce')
df_copy['average_rating'] = pd.to_numeric(df_copy['average_rating'], errors='coerce')
df_copy['isbn13'] = pd.to_numeric(df_copy['isbn13'], errors='coerce')
df_copy['num_pages'] = pd.to_numeric(df_copy['num_pages'], errors='coerce')
df_copy['ratings_count'] = pd.to_numeric(df_copy['ratings_count'], errors='coerce')
df_copy['text_reviews_count'] = pd.to_numeric(df_copy['text_reviews_count'], errors='coerce')

#  Convert publication_date to datetime if it's not already
if not pd.api.types.is_datetime64_any_dtype(df_copy['publication_date']):
    df_copy['publication_date'] = pd.to_datetime(df_copy['publication_date'], errors='coerce')

# Replace NaN with None for proper NULL handling in PostgreSQL
df_copy = df_copy.replace({pd.NA: None, pd.NaT: None})
df_copy = df_copy.where(pd.notnull(df_copy), None)

### Insert data into Books table

In [22]:
# Convert DataFrame to csv format in memory
df_copy.to_csv(output, sep='\t', header=False, index=False, na_rep='\\N')
output.seek(0)
with conn.cursor() as cur:
# Use COPY to insert data
    cur.copy_from(
        file=output,
        table='books',
        null='\\N'
    )

# Commit and close
conn.commit()
conn.close()

In [23]:
df_copy.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,2006-09-16 00:00:00,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,2004-09-01 00:00:00,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,2003-11-01 00:00:00,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,2004-05-01 00:00:00,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,2004-09-13 00:00:00,Scholastic


In [33]:
conn = create_db_connection()
with conn.cursor() as cur:
    cur.execute("""SELECT aidb.create_model('paraphrase', 
                            'bert_local', 
                            '{"model": "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
                             "revision": "main"}'::JSONB);""")
    cur.execute("""SELECT aidb.create_retriever_for_table(
            name => 'book_retriever_aidb',
            model_name => 'paraphrase',
            source_key_column => 'bookid',
            source_table => 'books',
            source_data_column => 'title',
            source_data_type => 'Text');""")
    
    cur.execute("SELECT aidb.bulk_embedding('book_retriever_aidb');")
    # Commit and close
conn.commit()


In [ ]:
# Retrieve books that their average_rating is over 4.5 and have at least 1000 reviews, initially. 
# Then, I want books with magic and dragon involved.

with conn.cursor() as cur:
    cur.execute("""WITH filtered_books AS (
    SELECT bookid
    FROM books
    WHERE text_reviews_count > 1000 
        AND average_rating > 4.5
),
query_embedding AS (
    SELECT aidb.encode_text('paraphrase', 'books with dragon and magic')::vector AS query_vec
)
SELECT br.id, b.title, b.authors
FROM book_retriever_aidb_vector br
JOIN books b ON b.bookid::text = br.id  -- Cast bookid to text to match br.id
WHERE br.id IN (SELECT bookid::text FROM filtered_books)  -- Cast bookid to text in the subquery
ORDER BY br.embeddings <=> (SELECT query_vec FROM query_embedding)
LIMIT 20;""")
    embeddings = cur.fetchall()
    # Print the embeddings in a table structure
    print(tabulate(embeddings, headers=["ID", "Title", "Authors"], tablefmt="pretty"))

+-------+-----------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------+
|  ID   |                                 Title                                 |                                                                    Authors                                                                    |
+-------+-----------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------+
|  30   | J.R.R. Tolkien 4-Book Boxed Set: The Hobbit and The Lord of the Rings |                                                                J.R.R. Tolkien                                                                 |
|   5   |      Harry Potter and the Prisoner of Azkaban (Harry Potter  #3)      |               

In [ ]:
conn.close()